In [ ]:
%autosave 0
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Die Onkologie-Station

Auf einer Onkologie-Station liegen fünf Patienten in nebeneinander liegenden Zimmern.
Bis auf einen  der Patienten hat jeder genau eine Zigaretten-Marke geraucht.
Der Patient, der nicht Zigarette geraucht hat, hat Pfeife geraucht.
Jeder Patient fährt genau ein Auto und ist
an genau einer Krebs-Art erkrankt.  Zusätzlich haben Sie die folgenden Informationen:
<ol>
<li> Im Zimmer neben Michael wird Camel geraucht. </li>
<li> Der Trabant-Fahrer raucht Ernte 23 und liegt im Zimmer neben dem 
      Zungen-Krebs Patienten. </li>
<li> Rolf liegt im letzten Zimmer und hat Kehlkopf-Krebs. </li>
<li> Der West-Raucher liegt im ersten Zimmer. </li>
<li> Der Mazda-Fahrer hat Zungen-Krebs und liegt neben dem Trabant-Fahrer. </li>
<li> Der Nissan-Fahrer liegt neben dem Zungen-Krebs Patient. </li>
<li> Rudolf wünscht sich Sterbe-Hilfe und liegt zwischen dem Camel-Raucher und dem Trabant-Fahrer. </li>
<li> Der Seat Fahrer hat morgen seinen letzten Geburtstag. </li>
<li> Der Luckies Raucher liegt neben dem Patienten mit Lungen-Krebs. </li>
<li> Der Camel Raucher liegt neben dem Patienten mit Darm-Krebs. </li>
<li> Der Nissan Fahrer liegt neben dem Mazda-Fahrer. </li>
<li> Der Mercedes-Fahrer raucht Pfeife und liegt neben dem Camel Raucher. </li>
<li> Jens liegt neben dem Luckies Raucher. </li>
<li> Der Hodenkrebs-Patient hat gestern seine Eier durchs Klo gespült. </li>
</ol>
Entwickeln Sie ein <em>Python</em>-Programm, das die folgenden Fragen beantwortet:
<ol> 
<li> Was raucht der Darmkrebs-Patient? </li>
<li> Was fährt Kurt für ein Auto? </li>
</ol>

## Importing the Necessary Modules

Our goal is to solve this puzzle by first coding it as a solvability problem of propositional logic and then to solve the resulting set of clauses using the algorithm of Davis and Putnam.

In [ ]:
import davisPutnam as dp

In order to be able to transform formulas from propositional logic into sets of clauses we import the module <tt>cnf</tt> which implements the function <tt>normalize</tt> that takes a formula and transforms it into a set of clauses.

In [ ]:
import cnf

In order to write formulas conveniently, we use the parser for propositional logic.

In [ ]:
import propLogParser as plp

Using the parser and the module <tt>cnf</tt> we can impement a function $\texttt{parseCNF}(s)$ that takes a string $s$ representing a formula and transforms $s$ into an equivalent set of clauses.

In [ ]:
def parseCNF(s):
    nestedTuple = plp.LogicParser(s).parse()
    Clauses     = cnf.normalize(nestedTuple)
    return Clauses

## Auxiliary Functions

In [ ]:
def atMostOne(V):
    return { frozenset({('¬',p), ('¬', q)}) for p in V
                                            for q in V 
                                            if  p != q 
           }

Given a name $f$ and an index $i \in\{1,2,3,4,5\}$, the function $\texttt{var}(i)$ creates the string 
$f\langle i \rangle$, e.g. the call <tt>var("Japanese", 2)</tt> returns the following string:

<tt>Japanese$\langle$2$\rangle$</tt>.

In [ ]:
def var(f, i):
    return f + "<" + str(i) + ">" 

In [ ]:
var("Japanese", 2)

The call $\texttt{flatten}(\texttt{LoS})$ takes list of sets $\texttt{LoS}$ and adds all the sets in this list into one big set. 

A call of the form $\texttt{x}$ will return a clause that specifies that the person with property $x$ has to live in one of the houses from $1$ to $5$.

In [ ]:
Clauses = allClauses()
Clauses

In [ ]:
len(Clauses)

In [ ]:
def solve():
    Clauses = allClauses()
    return dp.solve(Clauses, set())

In [ ]:
import time

Solving the problem takes about 7 seconds on my computer.

In [ ]:
start    = time.time()
Solution = solve()
stop     = time.time()
print(f'Time needed: {round((stop-start)*10)/10} seconds.')
Solution

## Functions to PrettyPrint the Solution

In [ ]:
def arb(S):
    for x in S:
        return x

In [ ]:
def pad(s, l):
    n = l - len(s)
    return s + " " * n

In [ ]:
def join(L, sep):
    result = ''
    for s in L:
        result += s + sep
    if len(L) > 0:
        result = result[:-len(sep)]
    return result

In [ ]:
def printSolution(UnitClauses):
    Brands     = { "Camel", "Ernte", "West", "Luckies", "Pfeife"    }
    Cars       = { "Trabant", "Mazda", "Nissan", "Seat", "Mercedes" }
    Cancers    = { "Zunge", "Kehlkopf", "Lunge", "Darm", "Hoden" }
    Names      = { "Michael", "Rolf", "Rudolf", "Jens", "Kurt" }
    Assignment = {}
    CarsDict   = {}
    BrandsDict = {}
    for Unit in UnitClauses:
        Literal = arb(Unit)
        if isinstance(Literal, str):
            number = int(Literal[-2])
            name   = Literal[:-3]
            Assignment[name] = number
            if name in Brands:
                BrandsDict[number] = name
            if name in Cars:
                CarsDict[number] = name
    print()

    print(f"Der Darmkrebs-Patient raucht {BrandsDict[Assignment['Darm']]}.");
    print(f"Kurt fährt einen {CarsDict[Assignment['Kurt']]}.");
    print()
    longest = max({ len(x) for x in Assignment })
    line    = "-" * (4 * (longest + 3) + 6);
    for house in range(1, 5+1):
        print(line)
        l = [];
        for Class in [Brands, Cars, Cancers, Names]:
            for x in Class:
                l += [ pad(f"{y}", longest) for y in Assignment 
                                            if y == x and Assignment[y] == house
                     ]
        print(f"| {house}: | " + join(l, " | ") + " |")
    print(line)
    print()

In [ ]:
printSolution(Solution)

## Checking the Uniqueness of the Solution

Given a set of unit clauses $U$, the function $\texttt{checkUniqueness}(U)$ returns a clause that is the negation of the set $U$.

In [ ]:
def negateSolution(UnitClauses):
    return { dp.complement(arb(unit)) for unit in UnitClauses }

In [ ]:
negateSolution({ frozenset({'a'}), frozenset({('¬', 'b')}) }) 

The function $\texttt{checkUniqueness}(\texttt{Solution}, \texttt{Clauses})$  takes a set of $\texttt{Clauses}$ and a $\texttt{Solution}$ for these clauses and checks, whether this is the only solution.

In [ ]:
def checkUniqueness(Solution, Clauses):
    negation = negateSolution(Solution)
    Clauses.add(frozenset(negation))
    alternative = dp.solve(Clauses, set())
    if alternative == { frozenset() }:
        print("Well done: The solution is unique!")
    else:
        print("ERROR: The solution is not unique!")

In [ ]:
checkUniqueness(Solution, Clauses)